In [730]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [731]:
images = [cv2.imread("../mire_1.png"), cv2.imread("../mire_2.png")]


Nous allons adopter les conventions suivantes : les axes x,y sont tels que x correspond à la coordonnée en ligne du pixel et y correspond à la coordonnée en colonne du pixel, ainsi Nx correspond au nombre de corners en "ordonnée", Ny correspond au nombre de coordonnées en "abscisse".

In [732]:
cnt = 1
ny = 11
nx = 8
coord_px = []
for image in images:
    ret, corners = cv2.findChessboardCorners(image, (ny, nx))
    cv2.drawChessboardCorners(image, (ny, nx), corners, ret)
    result_name = '../data/board_corners_draw'+str(cnt)+'.jpg'
    cv2.imwrite(result_name, image)
    # if cnt == 1:
        # plt.imshow(image)
    coord_px.append(np.flip(np.squeeze(corners)))
    cnt += 1
coord_px = np.concatenate((coord_px[0], coord_px[1]))

In [733]:
ecart_corners_x = np.linspace(0, 20*nx - 20, nx)
ecart_corners_y = np.linspace(0, 20*ny - 20, ny)
ecart_corners_z_picture_1 = np.linspace(0, 0, ny)
ecart_corners_z_picture_2 = np.linspace(-100, -100, ny)

yv, xv = np.meshgrid(ecart_corners_y, ecart_corners_x)
zv_1, xv = np.meshgrid(ecart_corners_z_picture_1, ecart_corners_x)
zv_2, xv = np.meshgrid(ecart_corners_z_picture_2, ecart_corners_x)

coord_mm = np.stack((xv, yv, zv_1), axis=2) # La coordonnée selon z est 0
coord_mm = np.concatenate((coord_mm, np.stack((xv, yv, zv_2), axis=2)), axis=0)
coord_mm = coord_mm.reshape(176, 3)

D'après le site du constructeur de webcam logitech, la référence C270 a une distance focale de 4.0 mm.

In [734]:
f = 4
height, width = images[0].shape[0]/2, images[0].shape[1]/2
i1 = height
i2 = width

In [735]:
u_1_tilde = coord_px[:,0] - height # x 
u_2_tilde = coord_px[:,1] - width # y

In [736]:
lignes = 176

In [737]:
A = np.zeros((lignes, 7))
U1 = u_1_tilde

In [738]:
for i in range(lignes):
    A[i] = np.array([u_2_tilde[i]*coord_mm[i,0], u_2_tilde[i]*coord_mm[i,1], u_2_tilde[i]*coord_mm[i,2], u_2_tilde[i], -u_1_tilde[i]*coord_mm[i,0], -u_1_tilde[i]*coord_mm[i,1], -u_1_tilde[i]*coord_mm[i,2]])

In [739]:
L = np.linalg.inv(A.T @ A) @ A.T @ U1

Comme notre axe z est orienté à l'opposé de la direction des mires on choisit o2c comme une valeur négative.

In [740]:
o_2_c_abs = 1/(np.sqrt(L[4]**2+L[5]**2+L[6]**2))
o_2_c = o_2_c_abs
print(o_2_c)
beta = o_2_c_abs * np.sqrt(L[0]**2+L[1]**2+L[2]**2)
o_1_c = L[3] * o_2_c / beta
r11 = L[0] * o_2_c / beta
r12 = L[1] * o_2_c / beta
r13 = L[2] * o_2_c / beta
r21 = L[4] * o_2_c
r22 = L[5] * o_2_c
r23 = L[6] * o_2_c


97.53726346349515


In [741]:
R1 = np.array([r11, r12, r13])
R2 = np.array([r21, r22, r23])
R3 = np.cross(R1, R2)
r31, r32, r33 = R3[0], R3[1], R3[2]

In [742]:
phi = - np.arctan(r23/r33)
gamma = - np.arctan(r12/r11)
omega = np.arctan(r13/(-r23*np.sin(phi)+r33*np.cos(phi)))

In [743]:
B = np.zeros((lignes, 2))
R = np.zeros((lignes, 1))
for i in range(lignes):
    R[i] = -u_2_tilde[i] * (r31*coord_mm[i, 0]+r32*coord_mm[i, 1]+r33*coord_mm[i, 2])
    B[i] = np.array([u_2_tilde[i], -(r21*coord_mm[i, 0] + r22*coord_mm[i, 1] + r23*coord_mm[i, 2] + o_2_c)])

In [744]:
F = np.squeeze(np.linalg.pinv(B.T @ B) @ B.T @ R)
o_3_c = F[0]
f2 = F[1]
print(F)

[-332.30223876  801.52607047]


In [745]:
s2 = f/f2
f1 = beta * f2
s1 = f/f1
print(f2)
print(f)
print(s1)
print(s2)

801.5260704660341
4
<class 'numpy.float64'>
0.004990480219407294


In [756]:
M_int = np.array(
    [[f/s1, 0, i1, 0],
     [0, f/s2, i2, 0],
     [0, 0, 1, 0]]
)
M_ext = np.array(
    [[r11, r12, r13, o_1_c],
     [r21, r22, r23, o_2_c],
     [r31, r32, r33, o_3_c],
     [0, 0, 0, 1]]
)
M = M_int @ M_ext

In [760]:
im = plt.imread("mire_1.png")
fig, ax = plt.subplots()
x = np.ndarray((88,1))
for i in range(int(lignes/2)):
    alpha_U = M @ np.append(coord_mm[i, :], 1)
    alpha = alpha_U[2]
    U = alpha_U / alpha

plt.show()


[38.72786164 84.73628406  1.        ]
[ 41.36089757 133.76227445   1.        ]
[ 43.96545059 182.25792526   1.        ]
[ 46.54198038 230.23179564   1.        ]
[ 49.09093678 277.69226154   1.        ]
[ 51.61276007 324.64752058   1.        ]
[ 54.10788118 371.10559679   1.        ]
[ 56.576722   417.07434517   1.        ]
[ 59.01969555 462.56145611   1.        ]
[ 61.43720624 507.5744597    1.        ]
[ 63.8296501  552.12072987   1.        ]
[83.62291682 81.06309213  1.        ]
[ 86.03936282 130.67483039   1.        ]
[ 88.42936899 179.74373574   1.        ]
[ 90.79336689 228.27866893   1.        ]
[ 93.13177878 276.28829889   1.        ]
[ 95.44501779 323.7811079    1.        ]
[ 97.73348824 370.7653966    1.        ]
[ 99.9975858  417.24928884   1.        ]
[102.23769779 463.24073633   1.        ]
[104.45420334 508.74752318   1.        ]
[106.64747365 553.77727031   1.        ]
[129.57170254  77.30368683   1.        ]
[131.76063921 127.5153243    1.        ]
[133.92534741 177.1711